In [15]:
import numpy as np

In [35]:
import tensorflow.contrib.keras as kr

In [11]:
import fasttext

In [1]:
import tensorflow as tf

In [13]:
def loadWordEmbedding(model):
    model = fasttext.load_model(model)
    vocab = ['unk']
    embd = [[0] * 100]
    for word in model.words:
        vocab.append(word)
        embd.append(model[word])
    print("loaded word2vec")
    return vocab, embd

In [16]:
vocab, embd = loadWordEmbedding('model/fasttext.model.bin')
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

loaded word2vec


In [13]:
vocab_size

226403

 **tensorflow load the trained word embedding (fasttext)**<br >
 return the vocab and embedding<br>
 the embedding_dim is 100<br>
 the vocab: 226403
 - 'unk',
 - '处会',
 - '倍觉',
 - '螳螂捕蝉',
 - '烟标',
 - '见证人',
 - '玩到',
 - '刑满释放',
 
the embedding:
- [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
     0.        ,  0.        ],
- [ 0.20675033,  0.13941619, -0.02178967, ...,  0.08394553,
    -0.57525939, -0.17493315],
- [ 0.02645796, -0.16807704, -0.33025488, ...,  0.18632847,
    0.20377228,  0.1036511 ],

note: add the unk is that the word not in trained word embedding. and the vector is 0

In [64]:
words = []
with open('PreData/train_cut', 'r', encoding='utf8') as f:
    for doc in f:
        words.append(len(doc.split()))

average = np.average(words)
word_max = np.max(words)
print(average), print(word_max)

316.191693819
1717


(None, None)

In [6]:
label = {}
with open('PreData/train', 'r', encoding='utf8') as f:
    for l_doc in f:
        l, _ = l_doc.split('\t')
        if l in label:
            label[l] += 1
        else:
            label[l] = 1
label_number = len(label)

In [8]:
label_number, label

(13,
 {'IT': 7021,
  '体育': 58025,
  '健康': 3486,
  '军事': 1906,
  '奥运': 18205,
  '女性': 11295,
  '娱乐': 22083,
  '房产': 37069,
  '教育': 6589,
  '文化': 2106,
  '旅游': 5945,
  '汽车': 4180,
  '财经': 40000})

**train data format**
- label /t document <br>
```
IT      互联网巨头雅虎公司26日宣布,公司正对高级管理层实施“大手术”,同时组建3个业务部门形成全新的公司组织构架。雅虎总裁苏珊·德克说,微软公司收购雅虎的谈判两周前结束后,雅虎终于可以专注于已耽搁数月的重组计划,为公司向赚钱机器转型迈出一大步。全新构架雅虎当天在声明中说,公司正在组建3个新业务部门,旨在把产品研发部门设为核心业务部门,同时加强产品研发部门与技术...
```

**train data cut format**
- words[0] space words[1]..... space words[end] <br>
```
来源 : 新华网 加拿大 多伦多市 一名 建筑师 推出 的 摩天 农场 概念 引起 世人 关注 。 有别于 传统 农场 , 摩天 农场 向 空中 延伸 , 形似 摩天大楼 。 如果 摩天 农场 落成 , 农作物 产出 每年 可 满足 约 3.5 万名 居民 的 需求 。 目前 , 世界 上 已 出现 数个 类似 的 ... 
```

- label number: 13
- detail of train_data:
```
 {'IT': 7021,
  '体育': 58025,
  '健康': 3486,
  '军事': 1906,
  '奥运': 18205,
  '女性': 11295,
  '娱乐': 22083,
  '房产': 37069,
  '教育': 6589,
  '文化': 2106,
  '旅游': 5945,
  '汽车': 4180,
  '财经': 40000}
```

note: **compute the average value of words length (seq_length)**


In [14]:
class RnnConfig(object):
    embedding_dim = 100
    seq_length = 320
    num_classes = 13
    vocab_size = 226403
    
    num_layer = 2
    hidden_dim = 128
    rnn = 'lstm'
    
    dropout_keep_prob = 0.5
    learning_rate = 1e-3
    
    batch_size = 128
    num_epochs = 10
    
    print_per_batch = 100
    save_per_batch = 10

In [3]:
class TextRnn(object):
    def __init__(self, config):
        self.config = config
        
        self.input_x = tf.placeholder(tf.float32, [None, self.config.seq_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, self.config.num_classes], name='input_y')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
        self.cnn()
    
    def rnn(self):
        def lstm_cell():
            return tf.contrib.rnn.BasicLSTMCell(self.config.hidden_dim, state_is_tuple=Truee)
        
        def dropout():
            cell = lstm_cell()
            return tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=sel.keep_prob)
        
        with tf.device('/gpu:0'):
            W = tf.Variable(tf.constant(0.0, shape[self.config.vocab_size, self.config.embedding_dim]), trainable=False, name='W')
            embedding_placeHolder = tf.placeholder(tf.float32, [self.config.vocab_size, self.config.embedding_dim])
            embedding_init = W.assign(embedding_placeHolder)
            
        with tf.name_scope("rnn"):
            cells = [dropout() for _ in range(self.config.num_layer)]
            rnn_cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)
            _outputs, _ = tf.nn.dynamic_rnn(cell=rnn_cell, inputs=embedding_init, dtype=tf.float32)
            last = _outputs[:, -1, :]
            
        with tf.name_scope("score"):
            fc = tf.layers.dense(last, self.config.hidden_dim, name='fc1')
            fc = tf.contrib.layers.dropout(fc, self.keep_prob)
            fc = tf.nn.relu(fc)
            
            self.logits = tf.layers.dense(fc, self.config.num_classes, name='fc2')
            self.y_pred_cls = tf.argmax(tf.nn.softmax(self.logits), 1)
            
        with tf.name_scope("optimize"):
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            self.loss = tf.reduce_mean(cross_entropy)
            self.optim = tf.train.AdamOptimizer(learning_rate=self.config.learning_rate).minimize(self.loss)
            
        with tf.name_scope("accuracy"):
            correnct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
            self.acc = tf.reduce_mean(tf.cast(correnct_pred, tf.float32))
            

In [7]:
def read_category():
    """
    categories transform to value 
    """
    categories = ['IT', '体育', '健康', '军事', '奥运', '女性', '娱乐', '房产', '教育', '文化', '旅游', '汽车','财经']
    cat_to_id = dict(zip(categories, range(len(categories))))
    return categories, cat_to_id

In [8]:
categories, cat_to_id = read_category()

In [20]:
word_to_id = dict(zip(vocab, range(vocab_size)))
word_to_id

{'盖房子': 1,
 '而书': 3,
 '其他': 4,
 '平房': 5,
 '要放': 6,
 '标本兼治': 7,
 '失魂': 8,
 '姚晓明': 9,
 '银票': 10,
 '斑秃': 11,
 '份量': 12,
 '四证': 13,
 '小剂量': 15,
 '非法收入': 16,
 '拉吊': 17,
 '季铭义': 18,
 '葛培健': 2,
 '刘能': 21,
 '幸运': 22,
 '拍拖': 23,
 '作曲者': 24,
 '无怪乎': 25,
 '侯世科': 26,
 '过硬': 27,
 '足协杯': 28,
 '塔里木': 75494,
 '时可据': 29,
 '糖分': 30,
 '对金': 31,
 '开车': 32,
 '回击': 150710,
 '答记者问': 34,
 'Montblanc': 35,
 '梅森': 75498,
 'G31201': 37,
 '活钱': 38,
 '对外商': 75499,
 '手机用户': 37606,
 '博望': 39,
 '新庄': 40,
 '失明': 41,
 '信奉者': 43,
 '建筑材料': 118624,
 '东台': 44,
 '迪亚兹': 45,
 '豪则': 46,
 '熊敏': 47,
 '血爆': 48,
 '名菜': 42362,
 '瞎掰': 49,
 '1520': 50,
 '万怡': 52,
 '滑稽地': 113317,
 '十七八岁': 53,
 '16.59%': 54,
 '13366106695': 55,
 '副校长': 56,
 '还刻': 58,
 '包件': 188302,
 '学徒': 59,
 '杜德克': 189833,
 '许小哈林': 188305,
 '锁链': 61,
 '虱子': 62,
 '秦兵': 63,
 'none': 64,
 '以资': 14023,
 '人南': 65,
 '甲状腺': 67,
 '13146848896': 68,
 '果色': 75,
 '儿童剧': 70,
 '性史': 71,
 '砼': 72,
 '王欢': 73,
 '高保': 74,
 '一马姓': 77,
 '洪天祥': 76,
 '已拉起': 208422,
 '张伟杰': 80,
 '扎沃德尼': 7

- **traisform to id**

    **the y_label is like this:**
        ```
         {'IT': 0,
          '体育': 1,
          '健康': 2,
          '军事': 3,
          '奥运': 4,
          '女性': 5,
          '娱乐': 6,
          '房产': 7,
          '教育': 8,
          '文化': 9,
          '旅游': 10,
          '汽车': 11,
          '财经': 12})
        ```

    **the x_data is like this:**
        ```
        {'盖房子': 1,
         '而书': 3,
         '其他': 4,
         '平房': 5,
         '要放': 6,
         '标本兼治': 7,
         '失魂': 8,
         '姚晓明': 9,
         '银票': 10,
         '斑秃': 11,
         '份量': 12,
         '四证': 13,
         '小剂量': 15,
        ```

In [21]:
def read_file(filename):
    with open(filename, 'r', encoding='utf8') as f:
        for line in f:
            yield line

In [ ]:
def process_file(content_dir, label_dir, word_to_id, cat_to_id, max_length=1000):
    '''
    make the train data
    '''
    contents = read_file(content_dir)
    labels = read_file(label_dir)
    
    data_id = [[word_to_id[x] for x in content.split() if x in word_to_id] for content in contents]
    label_id = [cat_to_id[label[:-1]] for label in labels]
    
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, 1000, padding='post', truncating='post')
    y_pad = kr.utils.to_categorical(label_id, num_classes=len(cat_to_id))
    

In [22]:
contents = read_file('./PreData/train_cut')
labels = read_file('./PreData/label_train')

In [23]:
data_id = [[word_to_id[x] for x in content.split() if x in word_to_id] for content in contents]

In [28]:
%%time
label_id = [cat_to_id[label[:-1]] for label in labels]

CPU times: user 49.8 ms, sys: 0 ns, total: 49.8 ms
Wall time: 60 ms


In [75]:
%%time
x_pad = kr.preprocessing.sequence.pad_sequences(data_id, 1000, padding='post', truncating='post')
y_pad = kr.utils.to_categorical(label_id, num_classes=len(cat_to_id))

CPU times: user 3.53 s, sys: 667 ms, total: 4.19 s
Wall time: 4.19 s


**make the train data to index and pad in same length**
- if the word vector size is smaller than specified length, then add the 0 (mode=post)
- else delete the more word (mode=post)

```
([  5607, 215414,  15425,  19772,  52617, 111590,  19585,  40881,
        86058, 121047, 147869, 204886, 216120, 154775, 207544, 218800,
        81033, 130909, 167262,  30992, 139440,  19772,  80670, 193948,
       135063,  31337, 213472, 215414, 198527,  40727, 178484,  38815,
       187325, 177281,  83589, 178484,  38815,  39448, 210307, 180697,
       177281, 216383, 116715,  42935,  85169, 207544, 218800,  44149,
       197243, 177281, 130909, 207544, 218800, 154775, 160307, 111590,
        81033,  43340,  70777,  12807,  15110, 177281, 123144,  63772,
        81033, 178484, 174865,  70777, 137332, 160307, 111590, 213472,
        71244, 177281,  19772,  52617, 100735,  81033,  56902, 211348,
        ...,
          0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0], dtype=int32
            
```

**make the category to index and transform to one-hot vector**

```
array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
```

In [76]:
print(' '.join(vocab[x] for x in x_pad[0]))
print()
print(' '.join(vocab[x] for x in data_id[0]))

早报 记者 周玲 中国联通 和 中国电信 将 在 7 月 25 日前 达成 出售 C 网 的 最终 协议 CFP 资料 中国联通 重组 正在 稳步 推进 。 记者 昨天 从 联通 集团 证实 , 昨天上午 联通 集团 召开 全国 电视电话会议 , 要求 即日起 全面 启动 C 网 资产 清查 , 最终 C 网 出售 给 中国电信 的 价格 可能 有点 变化 , 大约 三分之一 的 联通 员工 可能 会划 给 中国电信 。 此外 , 中国联通 和 中国网通 的 融合 也 在 稳步 推进 , 几天 前 网通 和 联通 各省市 分公司 的 一把手 全部 聚集 到 北京 开会 , 开始 初步 接触 , 联通 与 网通 成立 了 6 个 联合 工作组 开展 整合 相关 的 具体 工作 和 谈判 。 C 网将 出售 4 个 工作组 就 C 网 资产 清查工作 , 联通 集团 要求 6 月 25 日 完成 省级 分公司 清查工作 , 7 月 15 日 完成 总部 级 清查 , 中国联通 和 中国电信 将 在 7 月 25 日前 达成 最终 协议 , 60 日内 完成 资产 实质 交割 , 并 进行 第二阶段 付款 。 联通 集团 总部 有关 人士 向 记者 证实 , 上述 消息 属实 , 昨天上午 会议 主要 是 探讨 C 网 出售 的 资产 清查 和 人员 安排 。 中国联通 建华 昨天 对 记者 表示 , 由于 他 本人 没有 参加 昨天 的 电话会议 , 对 会议 的 细节 不 清楚 , 但是 联通 将 在 9 月份 召开 股东大会 对 C 网 出售 进行 表决 , 之前 围绕 C 网 资产 和 人员 应该 有 方案 。 昨天上午 的 会议 还 透露 , 联通 C 网 最终 的 出售 价格 将 按 C 网 2008 年 中报 的 实际 值 判定 , 如果 收入 低于 预期 98% , 每低 一个 百分点 , 降低 原 计划 交易 值 的 百分之一 。 按照 此前 , 中国联通 和 中国电信 达成 的 框架 协议 , 中国电信 上市公司 将 以 438 亿元 的 价格 收购 CDMA 业务 , 而 中国电信集团 以 662 亿元 的 价格 收购 CDMA 网络 资产 , 合计 1100 亿元 。 劳 建华 对此 解释 , 438 亿元 的 用户 资产 是 当初 双方 签署 的

In [ ]:
import os
import time

save_dir = './checkpoint/textRnn'

def train():
    print("Configuring TensorBoard and Saver...")
    tensorboard_dir = './textRnn'
    
    tf.summary.scalar('loss', model.loss)
    tf.summary.scalar('accuracy', model.acc)
    merged_summary = ft.summary.merged_all()
    writer = tf.summary.FileWriter(tensorboard_dir)
    
    saver = tf.train.Saver()
    
    print('Loading training and validation data...')
    start_time = time.time()
    x_train, y_train = process_file()
    

In [6]:
del(Out[5])